In [10]:
# !python -m pip install tables

In [281]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import scipy

In [12]:
store = pd.HDFStore('./variants.h5')

In [15]:
list(store.walk())

[('', ['Q_matrix'], ['variant_df', 'variant_q_df']), ('/Q_matrix', [], [])]

In [17]:
df = store.get("variant_df") 

In [73]:
df_strain = df

In [23]:
target = "Yield_Mg_ha"

In [19]:
df_trait = pd.read_csv("./1_trait_cleaned.csv")


In [55]:
df_trait[["Env", "Hybrid_Parent1", "Hybrid_Parent2"]].value_counts()

Env        Hybrid_Parent1   Hybrid_Parent2
MNH1_2019  B37              H95               19
NYH2_2019  MO44_PHW65_0319  PHT69             16
WIH2_2020  B73              PHN82             16
IAH3_2021  B73              MO17              16
IAH2_2018  2369             LH123HT           15
                                              ..
MIH1_2019  PHW65_MOG_0200   PHT69              1
           PHW65_MOG_0203   PHT69              1
           PHW65_MOG_0206   PHT69              1
           PHW65_MOG_0209   PHT69              1
WIH3_2021  W10004_1156      PHP02              1
Length: 84675, dtype: int64

In [92]:
def group_env_hybrid(df):
    res_row = df.iloc[0]
    res_row[target] = df[target].mean()
    return res_row
df_trait = df_trait.groupby(["Env", "Hybrid_Parent1", "Hybrid_Parent2"]).apply(group_env_hybrid).reset_index(drop=True)
df_trait.shape

(84675, 28)

In [94]:
genotyped_hybrids = set(df_strain.columns) 

In [96]:
df_trait_h_filtered = df_trait.loc[
    df_trait["Hybrid_Parent1"].isin(genotyped_hybrids) &\
    df_trait["Hybrid_Parent2"].isin(genotyped_hybrids)
]
df_trait_h_filtered.shape

(83300, 28)

In [97]:
df_trait_train = df_trait_h_filtered.loc[df_trait_h_filtered["Year"] < 2021]
df_trait_train = df_trait_train[["Env", "Hybrid_Parent1", "Hybrid_Parent2", target]].reset_index(drop=True)


In [123]:
def corr2_coeff(A, B):
    A_mA = A - A.mean(axis=1)[:, None]
    B_mB = B - B.mean(axis=1)[:, None]

    ssA = (A_mA**2).sum(axis=1)
    ssB = (B_mB**2).sum(axis=1)

    return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))

In [192]:
train_envs = df_trait_train["Env"].drop_duplicates()
correlations_1 = []
correlations_2 = []

for env in tqdm(train_envs):
    df_env = df_trait_train.query(f"Env == '{env}'")
    target_vector = df_env[target].to_numpy().reshape(1, -1)
    M = df_strain[df_env["Hybrid_Parent1"]].to_numpy()
    correlations = corr2_coeff(M, target_vector)
    correlations_1.append(correlations.flatten())
    M = df_strain[df_env["Hybrid_Parent2"]].to_numpy()
    correlations = corr2_coeff(M, target_vector)
    correlations_2.append(correlations.flatten())

  0%|          | 0/190 [00:00<?, ?it/s]

/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipyke

/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipyke

/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipyke

/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipyke

/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipyke

/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipyke

/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipyke

/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipykernel_2200031/3433548305.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
/tmp/ipyke

In [ ]:
correlations_2_df

In [193]:
correlations_1_df = pd.concat((pd.Series(c_v) for c_v in correlations_1), axis=1)

In [243]:
correlations_2_df = pd.concat((pd.Series(c_v) for c_v in correlations_2), axis=1)

In [255]:
correlations_1_df.to_csv("./correlations_parent_1.csv", index=False)
correlations_2_df.to_csv("./correlations_parent_2.csv", index=False)


KeyboardInterrupt



In [194]:
(correlations_1_df.abs() > 0.3).any(axis=1).sum()

267599

In [206]:
correlations_1_df.iloc[:, :10].abs().quantile(0.999, axis=0)

0    0.332586
1    0.249534
2    0.273213
3    0.263055
4    0.228848
5    0.266555
6    0.481264
7    0.281475
8    0.267657
9    0.372398
Name: 0.999, dtype: float64

In [232]:
k = 1000
top_indices = []
for i in tqdm(range(correlations_1_df.shape[1])):
    col = correlations_1_df[i].abs()
    tops = col.sort_values(ascending=False).index[:k]
    top_indices.append(tops)
len(set(np.concatenate(top_indices))) / (correlations_1_df.shape[1] * k)

  0%|          | 0/190 [00:00<?, ?it/s]

0.6508315789473684

In [244]:
k = 1000
top_indices_2 = []
for i in tqdm(range(correlations_2_df.shape[1])):
    col = correlations_2_df[i].abs()
    tops = col.sort_values(ascending=False).index[:k]
    top_indices_2.append(tops)
len(set(np.concatenate(top_indices_2))) / (correlations_2_df.shape[1] * k)

  0%|          | 0/190 [00:00<?, ?it/s]

0.6366315789473684

In [274]:
len(top_positions_2)

95

In [ ]:
top_k_rep = 65

In [275]:
for top_k_rep in range(50, 180, 5):
#     top_positions_1 = pd.Series(np.concatenate(top_indices)).value_counts().iloc[:top_k_rep].index
    top_positions_2 = pd.Series(np.concatenate(top_indices_2)).value_counts().iloc[:top_k_rep].index
    print(top_k_rep, df_strain.iloc[np.concatenate([top_positions_2]), :].transpose().drop_duplicates().shape[0])

50 1855
55 1877
60 1980
65 2093
70 2108
75 2137
80 2137
85 2146
90 2147
95 2149
100 2152
105 2152
110 2152
115 2154
120 2154
125 2161
130 2164
135 2165
140 2165
145 2168
150 2171
155 2177
160 2178
165 2179
170 2181
175 2182


In [272]:
df_strain.iloc[top_positions_1[:170], :].to_csv("./parent_1_mutations.csv", index=False)

In [276]:
df_strain.iloc[top_positions_2[:70], :].to_csv("./parent_2_mutations.csv", index=False)

In [ ]:
p_values_1 = []
p_values_2 = []
for env in tqdm(train_envs):
    df_env = df_trait_train.query(f"Env == '{env}'")
    target_vector = df_env[target].to_numpy().reshape(1, -1)
    n = df_env.shape[0]
    distribution = scipy.stats.beta(n/2 - 1, n/2 - 1, loc=-1, scale=2)
    p_s_1 = 2*(-correlations_1_df[0].abs()).map(distribution.cdf)
    p_values_1.append(p_s_1)
    
    p_s_2 = 2*(-correlations_2_df[0].abs()).map(distribution.cdf)
    p_values_2.append(p_s_2)


  0%|          | 0/190 [00:00<?, ?it/s]

In [309]:
p_values_1_df = pd.concat((pd.Series(p_v) for p_v in p_values_1), axis=1)
p_values_2_df = pd.concat((pd.Series(p_v) for p_v in p_values_2), axis=1)

In [314]:
(p_values_1_df < 0.01).any(axis=1).sum()

212168

In [317]:
n_envs = p_values_1_df.shape[1]

In [330]:
p_value_threshold = 0.01 / n_envs / p_values_1_df.shape[0]
(p_values_1_df < p_value_threshold).any(axis=1).sum()

41829

In [344]:
tops_1 = (p_values_1_df < p_value_threshold).sum(axis=1).sort_values(ascending=False)

In [345]:
tops_2 = (p_values_2_df < p_value_threshold).sum(axis=1).sort_values(ascending=False)

In [350]:
top_sep_positions_1 = []
last_n_unique = 0
for top_k_rep in range(1000):
    top_sep_positions_1.append(tops_1.index[top_k_rep])
    n_unique = df_strain.iloc[top_sep_positions_1].transpose().drop_duplicates().shape[0]
    if n_unique > last_n_unique:
        last_n_unique = n_unique
        print(
            top_k_rep,
            len(top_sep_positions_1),
            n_unique
        )
    else:
        top_sep_positions_1.pop()

0 1 2
1 2 4
2 3 6
3 4 10
4 5 19
5 6 28
7 7 36
10 8 50
11 9 60
14 10 77
15 11 80
16 12 81
18 13 83
19 14 105
21 15 107
25 16 126
26 17 136
27 18 137
28 19 144
29 20 146
30 21 147
32 22 157
33 23 161
35 24 182
38 25 229
40 26 230
43 27 231
44 28 239
48 29 248
52 30 256
53 31 263
55 32 271
56 33 274
57 34 275
64 35 279
65 36 280
66 37 284
68 38 288
71 39 304
74 40 310
77 41 327
78 42 328
82 43 333
84 44 341
85 45 358
86 46 361
87 47 381
88 48 388
89 49 423
90 50 424
91 51 426
92 52 428
94 53 454
96 54 461
97 55 466
99 56 500
100 57 503
101 58 538
102 59 551
104 60 553
105 61 558
106 62 561
112 63 593
113 64 615
114 65 616
115 66 627
116 67 633
118 68 662
120 69 666
122 70 675
124 71 676
125 72 678
126 73 683
127 74 696
129 75 763
138 76 771
139 77 772
142 78 774
143 79 780
145 80 783
149 81 796
150 82 797
151 83 852
152 84 855
155 85 856
156 86 865
159 87 868
161 88 871
163 89 874
165 90 906
169 91 907
171 92 908
173 93 911
174 94 958
178 95 971
179 96 972
180 97 977
184 98 978
186 99 979

In [351]:
top_sep_positions_2 = []
last_n_unique = 0
for top_k_rep in range(1000):
    top_sep_positions_2.append(tops_2.index[top_k_rep])
    n_unique = df_strain.iloc[top_sep_positions_2].transpose().drop_duplicates().shape[0]
    if n_unique > last_n_unique:
        last_n_unique = n_unique
        print(
            top_k_rep,
            len(top_sep_positions_2),
            n_unique
        )
    else:
        top_sep_positions_2.pop()

0 1 2
1 2 4
2 3 8
4 4 14
5 5 24
6 6 29
7 7 37
8 8 60
9 9 91
10 10 135
11 11 194
12 12 248
13 13 338
14 14 453
15 15 572
18 16 708
19 17 846
21 18 982
22 19 1020
23 20 1192
24 21 1290
25 22 1396
26 23 1482
27 24 1495
29 25 1604
36 26 1687
40 27 1779
41 28 1825
42 29 1827
47 30 1839
51 31 1840
53 32 1841
59 33 1864
60 34 1897
61 35 1938
63 36 1992
65 37 1995
67 38 2026
68 39 2030
69 40 2031
71 41 2048
72 42 2069
74 43 2083
88 44 2091
94 45 2093
95 46 2096
96 47 2103
99 48 2105
108 49 2129
109 50 2137
112 51 2138
114 52 2139
115 53 2140
121 54 2147
125 55 2148
131 56 2152
136 57 2157
142 58 2165
145 59 2166
150 60 2171
158 61 2172
163 62 2175
164 63 2177
165 64 2178
166 65 2180
189 66 2181
196 67 2182
203 68 2183
205 69 2184
236 70 2185
275 71 2186
279 72 2187
287 73 2189
294 74 2190
345 75 2192
348 76 2193
370 77 2194
389 78 2195
391 79 2196
444 80 2197
646 81 2198
762 82 2199
784 83 2200


In [352]:
df_strain.iloc[top_sep_positions_1[:227], :].to_csv("./parent_1_mutations_pv.csv", index=False)
df_strain.iloc[top_sep_positions_2[:83], :].to_csv("./parent_2_mutations_pv.csv", index=False)

In [354]:
df_strain.iloc[top_sep_positions_2[:83], :]

,2369,2FACC,4N506,6F629,740,A632,A634,A635,A641,A672,...,Z032E0081,Z033E0001,Z034E0042,Z034E0064,Z035E0047,Z035E0060,Z036E0057,Z037E0012,Z037E0054,Z038E0057
36759,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
84000,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
36755,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
52813,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
106586,1,0,0,0,1,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236478,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
74281,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
163660,0,0,0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
71808,0,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [348]:
len(top_sep_positions_1)

96

In [346]:
for top_k_rep in range(10, 180, 5):
    top_positions_1 = pd.Series(tops_1.iloc[:top_k_rep].index)
    top_positions_2 = pd.Series(tops_2.iloc[:top_k_rep].index)
    print(
        top_k_rep,
        df_strain.iloc[top_positions_1].transpose().drop_duplicates().shape[0],
        df_strain.iloc[top_positions_2].transpose().drop_duplicates().shape[0]
    )

10 36 91
15 77 453
20 105 846
25 107 1290
30 146 1604
35 161 1604
40 229 1687
45 239 1827
50 248 1839
55 263 1841
60 275 1864
65 279 1992
70 288 2031
75 310 2083
80 328 2083
85 341 2083
90 423 2091
95 454 2093
100 500 2105
105 553 2105
110 561 2137
115 616 2139
120 662 2140
125 676 2147
130 763 2148
135 763 2152
140 772 2157
145 780 2165
150 796 2166
155 855 2171
160 868 2172
165 874 2177
170 907 2180
175 958 2180


In [58]:
df_trait.query(f"Year == 2019 and Hybrid_Parent1 == 'B37' and Hybrid_Parent2 == 'H95' and Env == 'MNH1_2019'")[["Replicate", "Block", "Plot_Area_ha", target]]

,Replicate,Block,Plot_Area_ha,Yield_Mg_ha
80702,1,1,0.000883,9.828712
80734,1,1,0.000883,17.524738
80832,1,2,0.000883,5.797210
80899,1,3,0.000883,3.684297
80987,1,3,0.000883,4.113817
81116,1,4,0.000883,6.411487
81338,1,6,0.000883,10.413789
81393,1,6,0.000883,10.609834
81434,1,6,0.000883,11.382897
81482,2,7,0.000883,7.149140


In [42]:
df_trait_train[["Year", "Hybrid_Parent1", "Hybrid_Parent2", "Field_Location"]].value_counts()

Year  Hybrid_Parent1   Hybrid_Parent2  Field_Location
2019  B37              H95             MNH1              19
      MO44_PHW65_0319  PHT69           NYH2              16
2020  B73              PHN82           WIH2              16
                                       MOH1              15
2018  2369             LH123HT         IAH2              15
                                                         ..
      PHW65_MOG_0173   LH195           ILH1               1
2015  GEMS-0219        LH82            KSH1               1
                                       ILH1               1
2018  PHW65_MOG_0173   LH195           MNH1               1
2020  W10004_1156      PHZ51           WIH2               1
Length: 70490, dtype: int64

In [51]:
df_trait.query(f"Year == 2019 and Hybrid_Parent1 == 'B37' and Hybrid_Parent2 == 'H95' and Field_Location == 'MNH1'")[["Replicate", "Block", target]]

,Replicate,Block,Yield_Mg_ha
80702,1,1,9.828712
80734,1,1,17.524738
80832,1,2,5.797210
80899,1,3,3.684297
80987,1,3,4.113817
81116,1,4,6.411487
81338,1,6,10.413789
81393,1,6,10.609834
81434,1,6,11.382897
81482,2,7,7.149140
